In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('bank/bank-full.csv',sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df.shape

(45211, 17)

In [3]:
#get unique values in each column
for col in df.columns:
    print(col, df[col].unique())
    

age [58 44 33 47 35 28 42 43 41 29 53 57 51 45 60 56 32 25 40 39 52 46 36 49
 59 37 50 54 55 48 24 38 31 30 27 34 23 26 61 22 21 20 66 62 83 75 67 70
 65 68 64 69 72 71 19 76 85 63 90 82 73 74 78 80 94 79 77 86 95 81 18 89
 84 87 92 93 88]
job ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital ['married' 'single' 'divorced']
education ['tertiary' 'secondary' 'unknown' 'primary']
default ['no' 'yes']
balance [ 2143    29     2 ...  8205 14204 16353]
housing ['yes' 'no']
loan ['no' 'yes']
contact ['unknown' 'cellular' 'telephone']
day [ 5  6  7  8  9 12 13 14 15 16 19 20 21 23 26 27 28 29 30  2  3  4 11 17
 18 24 25  1 10 22 31]
month ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
duration [ 261  151   76 ... 1298 1246 1556]
campaign [ 1  2  3  5  4  6  7  8  9 10 11 12 13 19 14 24 16 32 18 22 15 17 25 21
 43 51 63 41 26 28 55 50 38 23 20 29 31 37 30 46 27

In [4]:
#remove rows with unknown values
df = df[(df['job'] != 'unknown') & (df['marital'] != 'unknown') & (df['education'] != 'unknown') & (df['default'] != 'unknown') & (df['housing'] != 'unknown') & (df['loan'] != 'unknown') & (df['contact'] != 'unknown') & (df['poutcome'] != 'unknown')]

In [5]:
for col in df.columns:
    print(col, df[col].value_counts())

age age
33    412
32    393
34    381
37    368
35    358
     ... 
81      3
86      3
18      2
89      1
88      1
Name: count, Length: 70, dtype: int64
job job
management       1753
blue-collar      1537
technician       1289
admin.           1057
services          682
retired           458
self-employed     264
student           237
entrepreneur      211
unemployed        208
housemaid         146
Name: count, dtype: int64
marital marital
married     4501
single      2454
divorced     887
Name: count, dtype: int64
education education
secondary    4197
tertiary     2633
primary      1012
Name: count, dtype: int64
default default
no     7786
yes      56
Name: count, dtype: int64
balance balance
0        432
1         20
2         20
5         17
4         17
        ... 
4121       1
1589       1
13156      1
9077       1
1475       1
Name: count, Length: 3090, dtype: int64
housing housing
yes    4942
no     2900
Name: count, dtype: int64
loan loan
no     6753
yes    1089
Name: coun

In [6]:
#drop multiple columns
df = df.drop(['balance','day','pdays','duration'],axis=1)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,campaign,previous,poutcome,y
24060,33,admin.,married,tertiary,no,no,no,telephone,oct,1,3,failure,no
24062,42,admin.,single,secondary,no,yes,yes,telephone,oct,1,1,other,yes
24064,33,services,married,secondary,no,yes,no,telephone,oct,1,4,failure,yes
24072,36,management,married,tertiary,no,yes,no,telephone,oct,1,4,other,no
24077,36,management,married,tertiary,no,yes,no,telephone,oct,1,3,failure,yes


In [7]:
df.shape

(7842, 13)

In [8]:
#re index the dataframe
df = df.reset_index(drop=True)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,campaign,previous,poutcome,y
0,33,admin.,married,tertiary,no,no,no,telephone,oct,1,3,failure,no
1,42,admin.,single,secondary,no,yes,yes,telephone,oct,1,1,other,yes
2,33,services,married,secondary,no,yes,no,telephone,oct,1,4,failure,yes
3,36,management,married,tertiary,no,yes,no,telephone,oct,1,4,other,no
4,36,management,married,tertiary,no,yes,no,telephone,oct,1,3,failure,yes


In [9]:
#use label encoder to convert categorical values to numerical values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in ['job','marital','education','default','housing','loan','contact','poutcome','y']:
    df[col] = le.fit_transform(df[col])
df.head()


,age,job,marital,education,default,housing,loan,contact,month,campaign,previous,poutcome,y
0,33,0,1,2,0,0,0,1,oct,1,3,0,0
1,42,0,2,1,0,1,1,1,oct,1,1,1,1
2,33,7,1,1,0,1,0,1,oct,1,4,0,1
3,36,4,1,2,0,1,0,1,oct,1,4,1,0
4,36,4,1,2,0,1,0,1,oct,1,3,0,1


In [10]:
#map month to numerical values
month_map = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
df['month'] = df['month'].map(month_map)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,campaign,previous,poutcome,y
0,33,0,1,2,0,0,0,1,10,1,3,0,0
1,42,0,2,1,0,1,1,1,10,1,1,1,1
2,33,7,1,1,0,1,0,1,10,1,4,0,1
3,36,4,1,2,0,1,0,1,10,1,4,1,0
4,36,4,1,2,0,1,0,1,10,1,3,0,1


In [11]:
for col in df.columns:
    print(col, df[col].unique())

age [33 42 36 44 26 51 30 34 49 47 40 38 54 52 48 45 39 37 43 35 50 56 59 53
 46 32 55 41 57 58 31 60 29 27 28 25 24 22 23 21 83 61 73 19 72 69 62 65
 71 64 20 67 76 68 63 70 74 79 75 77 82 66 80 18 78 81 84 89 86 88]
job [ 0  7  4  1  9 10  2  3  5  6  8]
marital [1 2 0]
education [2 1 0]
default [0 1]
housing [0 1]
loan [0 1]
contact [1 0]
month [10 11  1  2  3  4  5  6  7  8  9 12]
campaign [ 1  2  3  5  4  6  7  8  9 11 10 12 14 15 13 16]
previous [  3   1   4   2  16   6   5  10  12   7  18   9   8  11  14  15  26  37
  13  25  20  27  17  23  38  29  24  51 275  22  19  21  30  58  28  32
  40  55  35]
poutcome [0 1 2]
y [0 1]


In [12]:
sample_dict = {}
for i in range (10):
    sample_dict['S'+str(i+1)] = df.sample(frac=0.25,replace=False)

sample_dict

{'S1':       age  job  marital  education  default  housing  loan  contact  month   
 5125   38    9        2          1        0        0     0        0      5  \
 7763   77    5        0          0        0        0     0        0     10   
 3897   45    1        1          0        0        1     1        0      5   
 6564   32    9        1          1        0        1     0        0      2   
 2460   40    9        1          1        0        1     1        0      4   
 ...   ...  ...      ...        ...      ...      ...   ...      ...    ...   
 394    43    4        1          2        0        1     0        0     11   
 5304   30    4        2          2        0        0     0        0      6   
 3937   35    0        2          1        0        0     1        0      5   
 1589   31    9        2          0        0        1     0        0      2   
 6600   55    5        0          1        0        0     0        0      2   
 
       campaign  previous  poutcome  y  
 51

In [17]:
#import svm
#over sampler
from imblearn.over_sampling import SMOTE

import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
kernel_list = ['linear','poly','rbf','sigmoid']
best_kernel = ''
best_accuracy = 0
best_C = 0
best_gamma = 0
result_dict = {}

for key, value in sample_dict.items():
    sample_df = value
    X = sample_df.drop('y',axis=1)
    feature_name = X.columns
    y = sample_df['y']
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
    sm = SMOTE(random_state=2)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    for iter in range(1000):
        kernel = random.choice(kernel_list)
        C = random.uniform(0,1)
        gamma = random.uniform(0,1)
        model = SVC(kernel=kernel,C=C,gamma=gamma,max_iter=1)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test,y_pred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_kernel = kernel
            best_C = C
            best_gamma = gamma
    result_dict[key] = {'accuracy' : best_accuracy, 'kernel' : best_kernel, 'C' : best_C, 'gamma' : best_gamma}
    print('Best accuracy for sample ',key,' is ',best_accuracy)


    

Best accuracy for sample  S1  is  0.7602040816326531
Best accuracy for sample  S2  is  0.7772108843537415
Best accuracy for sample  S3  is  0.7874149659863946
Best accuracy for sample  S4  is  0.7925170068027211
Best accuracy for sample  S5  is  0.7925170068027211
Best accuracy for sample  S6  is  0.7925170068027211
Best accuracy for sample  S7  is  0.7925170068027211
Best accuracy for sample  S8  is  0.7925170068027211
Best accuracy for sample  S9  is  0.7925170068027211
Best accuracy for sample  S10  is  0.7925170068027211


In [19]:
print(result_dict)

{'S1': {'accuracy': 0.7602040816326531, 'kernel': 'sigmoid', 'C': 0.904418012754082, 'gamma': 0.12180412884757785}, 'S2': {'accuracy': 0.7772108843537415, 'kernel': 'rbf', 'C': 0.12481402749697545, 'gamma': 0.2470578918682892}, 'S3': {'accuracy': 0.7874149659863946, 'kernel': 'rbf', 'C': 0.7478396391447707, 'gamma': 0.032903381881532634}, 'S4': {'accuracy': 0.7925170068027211, 'kernel': 'rbf', 'C': 0.6713614451599669, 'gamma': 0.6276936114673513}, 'S5': {'accuracy': 0.7925170068027211, 'kernel': 'rbf', 'C': 0.6713614451599669, 'gamma': 0.6276936114673513}, 'S6': {'accuracy': 0.7925170068027211, 'kernel': 'rbf', 'C': 0.6713614451599669, 'gamma': 0.6276936114673513}, 'S7': {'accuracy': 0.7925170068027211, 'kernel': 'rbf', 'C': 0.6713614451599669, 'gamma': 0.6276936114673513}, 'S8': {'accuracy': 0.7925170068027211, 'kernel': 'rbf', 'C': 0.6713614451599669, 'gamma': 0.6276936114673513}, 'S9': {'accuracy': 0.7925170068027211, 'kernel': 'rbf', 'C': 0.6713614451599669, 'gamma': 0.627693611467